<a href="https://colab.research.google.com/github/jack978397/Colab-AI-Transcribe-Translate/blob/main/2_Gemini_Batch_Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-generativeai gradio
print("✅ 環境準備完成！")


In [ ]:
import gradio as gr
import google.generativeai as genai
import os
import time

# --- 1. 核心翻譯引擎 ---
class StandaloneTranslator:
    def __init__(self, api_key):
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel('gemini-3.1-pro-preview')

    def translate(self, text, src_lang, dst_lang):
        prompt = f"你是一位資深的{dst_lang}教學老師。請將以下 SRT 字幕精準翻譯為流暢、口語的{dst_lang}。規則：1.維持時間軸 2.保留引號 3.句尾不加標點 4.將 〇〇 換成 ～。\n\n內容：\n{text}"
        response = self.model.generate_content(prompt, generation_config=genai.types.GenerationConfig(temperature=0.1))
        return response.text

# --- 2. 處理邏輯 (支持實時更新) ---
def batch_translate_handler(api_key, files, src_lang, dst_lang, progress=gr.Progress()):
    if not api_key:
        yield "❌ 請填入 API Key", []
        return
    if not files:
        yield "❌ 請先上傳檔案", []
        return

    translator = StandaloneTranslator(api_key)
    log_content = "🚀 啟動全自動批次處理...\n"
    output_files = []

    for i, file_obj in enumerate(files):
        filename = os.path.basename(file_obj.name)
        log_content += f"正在翻譯 ({i+1}/{len(files)}): {filename}...\n"

        # 實時更新：讓畫面上出現「正在翻譯」的訊息
        yield log_content, output_files

        try:
            with open(file_obj.name, "r", encoding="utf-8") as f:
                content = f.read()

            # 呼叫 Gemini (這段會花比較多時間，請觀察 Colab 後台也有輸出)
            print(f"--- 正在發送至 Gemini 3.1: {filename} ---")
            translated = translator.translate(content, src_lang, dst_lang)

            out_name = f"translated_{dst_lang}_{filename}"
            with open(out_name, "w", encoding="utf-8") as f:
                f.write(translated)

            output_files.append(out_name)
            log_content += f"✅ {filename} 處理完畢！\n\n"

            # 實時更新：每翻完一個檔案就更新一次畫面
            yield log_content, output_files

        except Exception as e:
            log_content += f"❌ {filename} 發生錯誤: {str(e)}\n"
            yield log_content, output_files

    log_content += "--- 🎉 全部任務執行完畢 ---"
    yield log_content, output_files

# --- 3. 專屬 UI ---
with gr.Blocks(title="AI 字幕翻譯師") as translator_app:
    gr.Markdown("# 🌐 AI 專業字幕獨立翻譯系統 (實時日誌版)")
    api_key_input = gr.Textbox(label="🔑 Gemini API Key", type="password")

    with gr.Row():
        with gr.Column():
            file_input = gr.File(label="上傳 SRT 字幕檔 (支援多選)", file_count="multiple", file_types=[".srt"])
            src_lang = gr.Dropdown(["日文", "英文", "繁體中文"], label="來源語系", value="日文")
            dst_lang = gr.Dropdown(["繁體中文", "英文", "日文"], label="目標語系", value="繁體中文")
            btn = gr.Button("🚀 啟動全自動批次翻譯", variant="primary")
        with gr.Column():
            out_preview = gr.Textbox(label="實時處理日誌 (每完成一個檔案會更新一次)", lines=12, interactive=False)
            out_files = gr.File(label="📥 下載翻譯後的全部檔案")

    # 注意這裡的輸出綁定
    btn.click(batch_translate_handler, [api_key_input, file_input, src_lang, dst_lang], [out_preview, out_files])

translator_app.queue().launch(debug=True, share=True)
